# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [2]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.4834781749635813
 1.1591727103024008
 1.0766207657764593

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [3]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0      0.0
 0.277658  1.0      0.0
 0.23076   0.63745  1.0
U factor:
3×3 Matrix{Float64}:
 0.84346  0.288782  0.351236
 0.0      0.593553  0.15372
 0.0      0.0       0.257944

In [4]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [5]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [6]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0      0.0
 0.277658  1.0      0.0
 0.23076   0.63745  1.0

In [7]:
Alu.U

3×3 Matrix{Float64}:
 0.84346  0.288782  0.351236
 0.0      0.593553  0.15372
 0.0      0.0       0.257944

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [8]:
A\b

3-element Vector{Float64}:
 0.9999999999999999
 0.9999999999999998
 1.0000000000000007

In [9]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999999
 0.9999999999999998
 1.0000000000000007

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [10]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [11]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 -0.896746  -0.544161  -0.490826
  0.0       -0.662743  -0.298867
  0.0        0.0        0.217287

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [12]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}

In [13]:
Aqr.R

3×3 Matrix{Float64}:
 -0.896746  -0.544161  -0.490826
  0.0       -0.662743  -0.298867
  0.0        0.0        0.217287

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [14]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 0.35514699055827215
 1.0353628785794213
 2.517848647517045
vectors:
3×3 Matrix{Float64}:
 -0.143003   0.728467  -0.669989
 -0.516845  -0.632276  -0.577147
  0.844051  -0.263747  -0.466922

The values and the vectors can be extracted from the Eigen type by special indexing

In [15]:
AsymEig.values

3-element Vector{Float64}:
 0.35514699055827215
 1.0353628785794213
 2.517848647517045

In [16]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.143003   0.728467  -0.669989
 -0.516845  -0.632276  -0.577147
  0.844051  -0.263747  -0.466922

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [17]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0          -3.33067e-16  4.44089e-16
 -5.55112e-16   1.0          2.22045e-16
  1.55431e-15  -6.66134e-16  1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [18]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [19]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [20]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

1.9861882247716456

In [21]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [22]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [23]:
@time eigvals(Asym);

  1.400106 seconds (2.68 M allocations: 147.283 MiB, 1.49% gc time, 91.65% compilation time)


In [24]:
@time eigvals(Asym_noisy);

  0.810317 seconds (27 allocations: 7.928 MiB)


In [25]:
@time eigvals(Asym_explicit);

  0.166183 seconds (16.02 k allocations: 8.791 MiB, 18.45% gc time, 12.02% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [26]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.483765 seconds (1.04 M allocations: 237.374 MiB, 15.68% gc time, 40.56% compilation time)


6.73516756165448

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [27]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [28]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 3//5   1//5   7//10
 1//2   2//5    1
 1//10  3//10  1//5

In [29]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  3//2
 19//10
  3//5

In [30]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1
 1
 1

In [31]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
  1     0    0
 1//6   1    0
 5//6  7//8  1
U factor:
3×3 Matrix{Rational{BigInt}}:
 3//5  1//5    7//10
  0    4//15   1//12
  0     0     11//32

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [32]:
using LinearAlgebra

In [39]:
A_eigv = eigen(A)

OutOfMemoryError: OutOfMemoryError()

In [34]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

UndefVarError: UndefVarError: `A_eigv` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [36]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

UndefVarError: UndefVarError: `A_diag` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [38]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]

UndefVarError: UndefVarError: `A_lowertri` not defined in `Main`
Suggestion: check for spelling errors or missing imports.